In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from empyrical import max_drawdown, alpha, beta
from sklearn.preprocessing import MinMaxScaler
from src.twitter_interface import TwitterInterface
from src.asset_selector import AssetSelector
from src.indicators import Indicators
from util import time_formatter
from src.predictor import Predictor
from sklearn.cluster import KMeans
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import pandas as pd
import numpy as np
import configparser
import requests
import twitter
import json
import time
import sys
import os

weeks = 13

backdate = time_formatter(time.time() - (604800 * weeks))
config = configparser.ConfigParser()
try:
    config.read(os.path.relpath("config.ini"))
except FileExistsError as e:
    print("FileExistsError: {}".format(e))
    sys.exit(1)
alpaca_api = tradeapi.REST(
    base_url    = config["alpaca"]["APCA_API_BASE_URL"],
    key_id      = config["alpaca"]["APCA_API_KEY_ID"],
    secret_key  = config["alpaca"]["APCA_API_SECRET_KEY"],
    api_version = config["alpaca"]["VERSION"]
)
twitter_api = twitter.Api(
    config["twitter"]["CONSUMER_KEY"],
    config["twitter"]["CONSUMER_SECRET"],
    config["twitter"]["ACCESS_TOKEN_KEY"],
    config["twitter"]["ACCESS_TOKEN_SECRET"]
)
tq_key = config["tenquant"]["api_key"]

tdf = pd.DataFrame()
ti = TwitterInterface(twitter_api, tdf)
trading_account = alpaca_api.get_account()

selector = AssetSelector(alpaca_api, edgar_token=None)

def get_sentiment(text):
    """Given a text block, return a sentiment score based.
    :param text:
    :return:
    """
    text_polarity   = sid.polarity_scores(text)

    if text_polarity["compound"] > 0.50:
        sentiment   = "positive"
    else:
        sentiment   = "negative"
    return sentiment, text_polarity["compound"]

def get_losers():
    print('Losers'.center(45))
    print()
    print('Ticker'.ljust(10), 'Last'.ljust(11), 'Change'.ljust(11), 'Pct Loss')
    print("{:<30}".format('–' * 45))
    for symbol in range(len(alpaca_api.polygon.gainers_losers("losers"))):
        
        # print out some indicators here # TODO
        # also throw out shitty stocks
        
        print(alpaca_api.polygon.gainers_losers("losers")[symbol].ticker.ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers("losers")[symbol].lastTrade['p']).ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers("losers")[symbol].todaysChange).ljust(10),
              str(alpaca_api.polygon.gainers_losers("losers")[symbol].todaysChangePerc)+'%')

def get_gainers():
    print('Gainers'.center(45))
    print()
    print('Ticker'.ljust(10), 'Last'.ljust(11), 'Change'.ljust(11), 'Pct Gain')
    print("{:<30}".format('–' * 45))
    for symbol in range(len(alpaca_api.polygon.gainers_losers())):
        
        # print out some indicators here # TODO
        # also throw out shitty stocks
        
        print(alpaca_api.polygon.gainers_losers()[symbol].ticker.ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers()[symbol].lastTrade['p']).ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers()[symbol].todaysChange).ljust(10),
              str(alpaca_api.polygon.gainers_losers()[symbol].todaysChangePerc)+'%')

In [ ]:
if __name__ == '__main__':
    get_gainers()
    print()
    get_losers()
    print()
    print("Bulls".center(45))
    bulls = selector.bullish_candlesticks(64, 20)
    print()
    print("Bears".center(45))
    bears = selector.bearish_candlesticks(64, 20)

In [ ]:
ticker = "OSS"

In [ ]:
sid = SentimentIntensityAnalyzer()
stocktwits = requests.post("https://api.stocktwits.com/api/2/streams/symbol/{}.json".format(ticker))
stocktwits = json.loads(stocktwits.text)
stocktwitstext = "\n".join([m["body"] for m in stocktwits["messages"]])
tweets = ti.get_tweets(ticker, "stock")

In [ ]:
stsentiment, stscore = get_sentiment(stocktwitstext)
tsentiment, tscore = get_sentiment(tweets)

In [ ]:
print("Sentiment score for {}".format(ticker))
print("{:<30}".format('–' * 45))
print("Stocktwits:\t{}\t score: {}\nTwitter:\t{}\t score:\t{}".format(stsentiment, stscore, tsentiment, tscore))
print("Average:\t{}".format((stscore + tscore)/2))

In [ ]:
# alpaca_api.polygon.historic_trades(ticker, '2019-10-10', limit=10)
# alpaca_api.polygon.news(ticker)
# alpaca_api.polygon.last_trade(ticker)
# alpaca_api.polygon.last_quote(ticker)
# alpaca_api.polygon.company(ticker)
# alpaca_api.polygon.dividends(ticker)
# alpaca_api.polygon.financials(ticker)

In [ ]:
tenquant_data = requests.get("https://api.tenquant.io/data?key={}&ticker={}".format(tq_key, ticker))
tenquant_data = json.loads(tenquant_data.text)

In [ ]:
stocktwits

In [ ]:
tweets 

In [ ]:
tenquant_data